# Imports 

In [1]:
import os
from datasets import (
    load_dataset
)
import replicate
from langchain.llms import Replicate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

-------------

# Datasets

## Human Eval

In [2]:
dataset = load_dataset('openai_humaneval')

Found cached dataset openai_humaneval (/home/matheusalb/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)


  0%|          | 0/1 [00:00<?, ?it/s]

* Visualizing Example

In [3]:
print(dataset['test'][0]["prompt"])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



In [4]:
print(dataset['test'][0]['canonical_solution'])

    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False



# Inferance

## Model definition

In [5]:
os.environ['REPLICATE_API_TOKEN'] = 'r8_SFFfZ4rWUTbfw21S2MZXZVeTRuCkGjb06jw4Y'

# meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d
# meta/codellama-34b-instruct:b17fdb44c843000741367ae3d73e2bb710d7428a662238ddebbf4302db2b5422
# meta/codellama-34b-python:482ba325daab209d121f45a0030f2f3ed942df98b185d41635ab3f19165a3547
llm = Replicate(
    model="meta/codellama-7b:6c70f963b00e2186fa3048b9b6dba7c8f01ced30da78bd1853ae1b567282ebfb",
    model_kwargs={"temperature": 0, "max_tokens": 1024, "top_p": 0.1},
)

## Definição do Prompt

In [6]:
# template = """
# {context}
# {canonical_solution}

# def test():
# '''
# Test function of the above function. It contains 10 unit test cases for the above function.
# '''
# """.strip()


template = """
{context}
{canonical_solution}

# Write a test class with unittest for the above function. It should contain 10 unit test cases for the above function.
""".strip()

prompt = PromptTemplate(
    input_variables = ["context", "canonical_solution"],
    template=template,
)

chain = LLMChain(llm=llm, prompt=prompt)

## Executando Exemplo

In [7]:
# index= 70
# output = chain.run({'context': dataset['test'][index]['prompt'],
#            'canonical_solution': dataset['test'][index]['canonical_solution']})

In [8]:
# print(dataset['test'][index]['prompt']+ '\n'+ dataset['test'][index]['canonical_solution'] + '\n' + output)

In [9]:
# print(output);

In [10]:
outputs = []

print("Starting...")
print('-'*100)
for index in range(dataset['test'].num_rows):
    output = chain.run({'context': dataset['test'][index]['prompt'],
           'canonical_solution': dataset['test'][index]['canonical_solution']})
    
    print(output)
    print('-'*100)
    outputs.append([dataset['test'][index]['prompt'], dataset['test'][index]['canonical_solution'], output.strip()])

Starting...
----------------------------------------------------------------------------------------------------

import unittest


class TestTriplesSumToZero(unittest.TestCase):
    def test_triples_sum_to_zero(self):
        self.assertFalse(triples_sum_to_zero([1, 3, 5, 0]))
        self.assertTrue(triples_sum_to_zero([1, 3, -2, 1]))
        self.assertFalse(triples_sum_to_zero([1, 2, 3, 7]))
        self.assertTrue(triples_sum_to_zero([2, 4, -5, 3, 9, 7]))
        self.assertFalse(triples_sum_to_zero([1]))


if __name__ == '__main__':
    unittest.main()

----------------------------------------------------------------------------------------------------

class TestCarRaceCollision(unittest.TestCase):
    def test_car_race_collision(self):
        self.assertEqual(car_race_collision(2), 4)
        self.assertEqual(car_race_collision(3), 9)
        self.assertEqual(car_race_collision(4), 16)
        self.assertEqual(car_race_collision(5), 25)
        self.assertEqual(car_race_collis

In [11]:
len(outputs)

124

In [12]:
import pandas as pd
df = pd.DataFrame(outputs, columns=['prompt', 'canonical_solution', 'generated_solution'])

df.to_csv('generated_tests_codellama7b_replicate.csv', index=False)